In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/laurasisson/odor-pair'

In [18]:
import json
files = os.listdir("results")
data = []
for fname in files:
    if not fname.endswith(".json"):
        continue
    details = fname.split("_")
    if len(details)==2:
        experiment = {"concat":False,"size":details[0],"model":details[1][:-5]}
    else:
        experiment = {"concat":details[0],"size":details[1],"model":details[2][:-5]}
    with open(os.path.join("results",fname)) as f:
        experiment["results"] = json.load(f)
    data.append(experiment)
data[0]

{'concat': 'True',
 'size': 'small',
 'model': 'svmrbf',
 'results': {'citrus': 0.5909441227625568,
  'solvent': 0.31205542666292735,
  'oily': 0.4964746696597113,
  'licorice': 0.9754934915906482,
  'cheesy': 0.9936964232027539,
  'earthy': 0.5347763872670743,
  'acidic': 0.8314944539449944,
  'musty': 0.4961735275226424,
  'aromatic': 0.6067323886421222,
  'honey': 0.9814597726517351,
  'powdery': 0.39782756713051615,
  'orris': 0.26800584169865305,
  'fusel': 0.5820511583011584,
  'chocolate': 0.9502642642067927,
  'coconut': 0.8254977249267688,
  'balsamic': 0.8712327526802612,
  'tropical': 0.5345087105245798,
  'anise': 0.9660455179279789,
  'caramellic': 0.9066020868494852,
  'creamy': 0.7493306212360769,
  'coumarinic': 0.9246552356981805,
  'floral': 0.8148416945771652,
  'bitter': 0.719214753123141,
  'roasted': 0.6266385566443105,
  'phenolic': 0.3770504464927118,
  'fungal': 0.6992883268030519,
  'fatty': 0.6303269653738257,
  'sour': 0.9499800327971927,
  'green': 0.620253

In [6]:
descriptors = ["acidic","honey","mossy","sour","vanilla","amber","coffee","buttery","fresh","chocolate"] 

In [29]:
import pandas as pd
df = pd.DataFrame(data)
# Assuming df already has a 'results' column containing dictionaries
results_df = pd.json_normalize(df['results'])

# Combine the original df (without the 'results' column) with the exploded 'results' columns
df = df.drop(columns=['results']).join(results_df)
df

,concat,size,model,citrus,solvent,oily,licorice,cheesy,earthy,acidic,...,fresh,vegetable,ethereal,cocoa,tonka,chemical,waxy,minty,aldehydic,TOTAL
0,True,small,svmrbf,0.590944,0.312055,0.496475,0.975493,0.993696,0.534776,0.831494,...,0.979267,0.626891,0.673943,0.574613,0.774017,0.337553,0.761997,0.758439,0.866488,0.688864
1,True,big,svmrbf,0.676821,0.277623,0.527282,0.923476,0.993598,0.501688,0.878546,...,0.979983,0.694980,0.688183,0.533225,0.765737,0.078544,0.694680,0.834743,0.841372,0.707769
2,False,small,svmrbf,0.566384,0.432728,0.641005,0.923055,0.969392,0.559860,0.837468,...,0.970446,0.546846,0.628618,0.449690,0.867149,0.428208,0.725385,0.642144,0.849193,0.666164
3,True,big,svmlinear,0.573834,0.420031,0.485991,0.628661,0.959842,0.522086,0.898104,...,0.922417,0.587576,0.588344,0.270168,0.440566,0.085081,0.556741,0.649753,0.790282,0.608723
4,True,small,svmlinear,0.532223,0.483326,0.346197,0.661046,0.663791,0.581247,0.614243,...,0.687202,0.535715,0.495449,0.468256,0.815158,0.520057,0.560139,0.537707,0.710274,0.581412
5,False,small,lr,0.610846,0.395854,0.386016,0.695740,0.952690,0.571331,0.494583,...,0.538312,0.627227,0.504001,0.324094,0.625049,0.476564,0.582302,0.581264,0.649785,0.577972
6,False,big,rf,0.697763,0.478061,0.486551,0.781563,0.904845,0.439033,0.891079,...,0.995658,0.630527,0.634094,0.498590,0.639133,0.645748,0.766890,0.770984,0.801365,0.665318
7,False,big,svmrbf,0.642998,0.353238,0.566263,0.810365,0.993292,0.518988,0.909990,...,0.996706,0.668279,0.714443,0.388921,0.799224,0.099043,0.662039,0.756940,0.829669,0.685247
8,False,small,svmlinear,0.594528,0.508635,0.320915,0.607032,0.647188,0.591462,0.557481,...,0.597197,0.543040,0.433387,0.497177,0.564962,0.447233,0.568415,0.601899,0.645640,0.593012
9,True,big,lr,0.478323,0.463687,0.508598,0.455882,0.608417,0.492416,0.631916,...,0.302066,0.628224,0.499032,0.511954,0.467809,0.548948,0.512765,0.576835,0.492567,0.490645


In [37]:
model_cols = ["concat","size","model","TOTAL"]

In [47]:
best_rf = df[df["model"]=="rf"].sort_values("TOTAL",ascending=False)[model_cols]
best_rf_idx = best_rf.index[0]
best_rf

,concat,size,model,TOTAL
6,False,big,rf,0.665318
10,False,small,rf,0.659969


In [49]:
best_lr = df[df["model"]=="lr"].sort_values("TOTAL",ascending=False)[model_cols]
best_lr_idx = best_lr.index[0]
best_lr

,concat,size,model,TOTAL
5,False,small,lr,0.577972
12,False,big,lr,0.494145
9,True,big,lr,0.490645


In [50]:
best_svm = df[df["model"].str.contains("svm")].sort_values("TOTAL",ascending=False)[model_cols]
best_svm_idx = best_svm.index[0]
best_svm

,concat,size,model,TOTAL
1,True,big,svmrbf,0.707769
0,True,small,svmrbf,0.688864
7,False,big,svmrbf,0.685247
2,False,small,svmrbf,0.666164
3,True,big,svmlinear,0.608723
11,False,big,svmlinear,0.594011
8,False,small,svmlinear,0.593012
4,True,small,svmlinear,0.581412


In [68]:
top10 = ["TOTAL","acidic","honey","mossy","sour","vanilla","amber","coffee","buttery","fresh","chocolate"] 

In [73]:
idcs = [best_rf_idx,best_lr_idx,best_svm_idx]
results_df = df.iloc[idcs][["model"]+top10]
results_df[top10] = results_df[top10].map(lambda x: f'{x:,.2f}')
results_df

,model,TOTAL,acidic,honey,mossy,sour,vanilla,amber,coffee,buttery,fresh,chocolate
6,rf,0.67,0.89,0.88,0.77,0.49,0.93,0.91,0.78,0.81,1.00,0.54
5,lr,0.58,0.49,0.78,0.64,0.85,0.91,0.59,0.63,0.61,0.54,0.80
1,svmrbf,0.71,0.88,0.98,0.99,0.95,0.98,0.94,0.96,0.93,0.98,0.96


In [70]:
bottom10 = ["dairy","earthy","orris","cocoa","rummy","tropical","bitter","cooling","musty","sweet"]

In [74]:
results_df = df.iloc[idcs][["model"]+bottom10]
results_df[bottom10] = results_df[bottom10].map(lambda x: f'{x:,.2f}')
results_df

,model,dairy,earthy,orris,cocoa,rummy,tropical,bitter,cooling,musty,sweet
6,rf,0.49,0.44,0.49,0.50,0.50,0.49,0.47,0.50,0.56,0.57
5,lr,0.46,0.57,0.44,0.32,0.58,0.45,0.29,0.45,0.33,0.81
1,svmrbf,0.69,0.50,0.33,0.53,0.41,0.55,0.50,0.42,0.41,0.42
